import libraries and read data

In [ ]:
!pip uninstall eodal -y
!pip install git+https://github.com/lukasValentin/eodal.git@landsat-dev

In [ ]:
#import what is needed
from datetime import datetime
from pathlib import Path
from shapely.geometry import box

from eodal.config import get_settings
from eodal.core.sensors import Landsat
from eodal.core.scene import SceneCollection
from eodal.mapper.feature import Feature

In [ ]:
# read data from pickled file object into SceneCollectio
fpath = Path('Y:\MSc_23_TimckeFinn\data\EOdal\sample_mapper_data.pkl')
scoll = SceneCollection.from_pickle(stream=fpath)
scoll

delete no data

In [ ]:
%matplotlib inline  

# the scenes can be plotted
f_scenes = scoll.plot(['red', 'green', 'blue'])

In [ ]:
# loop over available scenes stored in scoll as a EOdal SceneCollection and check
# for no-data. These scenes will be removed from the SceneCollection
scenes_to_del = []
scoll.metadata['scene_used'] = 'yes'

for scene_id, scene in scoll:

    # check if scene is blackfilled (nodata); if yes continue
    if scene.is_blackfilled:
        scenes_to_del.append(scene_id)
        mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [blackfill]'
        continue

    # check cloud coverage (including shadows and snow) of the field parcel
    feature_cloud_cover = scene.get_cloudy_pixel_percentage(cloud_classes=[3, 8, 9, 10, 11])

    # if the scene is too cloudy, we skip it
    if feature_cloud_cover > feature_cloud_cover_threshold:
        scenes_to_del.append(scene_id)
        mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [clouds]'
        continue

    # calculate the NDVI and MSAVI
    scene.calc_si('NDVI', inplace=True)
    scene.calc_si('MSAVI', inplace=True)

# delete scenes too cloudy or containing only no-data
for scene_id in scenes_to_del:
    del mapper.data[scene_id]

Get values

In [ ]:
import numpy as np

# Specify the fill value
fill_value = 0

# Create a masked array
for scene_id, scene in scoll:
   
   masked_array = scene.get_values()

   # Convert masked array to a normal NumPy array with fill value
   normal_array = masked_array.filled(fill_value)

   # Convert NumPy array to Pandas DataFrame
   df = pd.DataFrame(normal_array)

print(df)

In [ ]:
str(scoll)